In [ ]:
import pandas as pd
import numpy as np
import warnings
import random
from multiprocessing import Manager
from concurrent.futures import ProcessPoolExecutor
warnings.filterwarnings("ignore")
from PIL import Image, ImageDraw, ImageFont
import operator
import docx
from docx.shared import Pt
import os
from balance_dfs import balance_dfs

available_players = ["nsilva", "guest_jpab", "guest_cgomes", "pduarte", 
                     "guest_diogo", "rpinheiro", "ggomes", "guest_pfreitas", 
                     "guest_gregs", 'guest_sono']

anygoal_data = pd.read_csv("anygoal_season_2023_resume.csv")

anybrain_df = pd.read_excel("players_07_23.xlsx", engine='openpyxl', sheet_name='Folha4')

player_stats = ['Ataque', 'Tecnica', 'Tatica', 'Defesa', 'Criatividade', 
                'Fisico', 'TrabalhoEquipa', 'Ambicao', 'GK', 'MeanPoints',
                'MeanPointsNoGR' 'Point_System']

In [ ]:
def add_guest_player(anybrain_df):
    # Get the new players
    new_players = set(available_players) - set(anybrain_df.Player.tolist())

    # Get the similar player
    similar_players = [input("Who does {} look like? ".format(player)) for player in new_players]
                         
    for player, guest in zip(similar_players, new_players):
        # Create a copy of the similar player
        player_data = anybrain_df[anybrain_df.Player == player].copy()
        # Change the player name
        player_data.loc[(player_data.Player == player), "Player"] = guest
        # Add the new row
        anybrain_df = anybrain_df.append(player_data.to_dict('records')[0], ignore_index=True)
        
        # Give any note on the player
        player_note = input("Any note for {}? ".format(guest)).upper().split()
        if any(x not in player_note for x in ["NO", "NONE", "NEGATIVE"]):
            for stat in player_stats[:-3]:
                # Create boolean masks
                better_stat_mask = (anybrain_df.Player == guest) & (stat.upper() in player_note) & ('better'.upper() in player_note)
                worse_stat_mask = (anybrain_df.Player == guest) & (stat.upper() in player_note) & ('worse'.upper() in player_note)

                # Stat update
                anybrain_df.loc[better_stat_mask & ('much'.upper() in player_note), stat] += 2
                anybrain_df.loc[worse_stat_mask & ('much'.upper() in player_note), stat] -= 2
                anybrain_df.loc[better_stat_mask, stat] += 1
                anybrain_df.loc[worse_stat_mask, stat] -= 1

                
        # Update the overall values
        aux = anybrain_df[(anybrain_df.Player == guest)]
        anybrain_df.loc[(anybrain_df.Player == guest), 'Overall'] = aux[player_stats[:-3]].mean(axis=1)
        anybrain_df.loc[(anybrain_df.Player == guest), 'Overall_No_GK'] = aux[player_stats[:-4]].mean(axis=1)
    
    return anybrain_df, list(new_players)
        
def treat_and_merge(anybrain_df, anygoal_data):
    anybrain_df = anybrain_df.dropna(axis=1, how='all')
    anybrain_df.columns = ['Player','Ataque', 'Tecnica', 'Tatica', 'Defesa', 'Criatividade', 
                           'Fisico', 'TrabalhoEquipa', 'Ambicao', 'GK', 'MeanPoints','MeanPointsNoGR']
    
    anybrain_df.Player.replace({'Jpab' : 'guest_jpab', 'J lopes' : 'guest_jlopes', 'carlos' : 'guest_cgomes', 
                              'Mineiro' : 'nsilva', 'Nelo' : 'esousa', 'Ze' : 'jsilva', 'Fintas' : 'spinto', 
                              'Pimenta' : 'apimenta', 'Xerife' : 'rpinheiro', 'Champion' : 'pduarte', 
                              'Gus' : 'ggomes', 'Costeira' : 'gcosteira', 'Flash' : 'guest_flash', 
                              'random' : 'guest_random', "anguyen" : 'ahn-vu', "gregs" : "guest_gregs"}, inplace=True)
    
    # Add the guest player to the bd, and his values will be the most similar to a specific player
    anybrain_df, new_players = add_guest_player(anybrain_df)
    new_players.extend([x for x in available_players if (x not in anygoal_data.Player.unique())])
    for player in np.unique(new_players):
        if (player not in anygoal_data.Player.unique()):
            anygoal_data = anygoal_data.append({'Player' : "{}".format(player), 'Participation' : 0, 'MVP' : 0, 
                                                'Assists' : 0, 'Result' : 0, 'BGK' : 0,
                                                'Goals' : 0, 'Point_System' : 0}, ignore_index = True)

    # Merge the anybrain_df with anygoal
    merged_df = anybrain_df.merge(anygoal_data, left_on='Player', right_on='Player')
    merged_df.drop(['Unnamed: 0',"Participation", "Goals", 
                    "Assists", "Own_Goals", "Result", "Overall", "Overall_No_GK", "MVP", "BGK"], axis=1, inplace=True)
    
    return merged_df

In [ ]:
merged_df = treat_and_merge(anybrain_df, anygoal_data)

In [ ]:
data_players = merged_df[merged_df.Player.isin(available_players)]
data_players

In [ ]:
def parallel_algorithm(df, n_process=8):
    # Split the iterations between the available processes
    counter = int(450000 / n_process)
    
    # Create a list that will contain the final dataframe
    final_list = Manager().list()
    
    seeds = [random.randint(1, 25) for _ in range(n_process)]
    
    with ProcessPoolExecutor(max_workers=n_process) as executor:
        futures = [executor.submit(balance_dfs, df, seed, counter) for seed in seeds]
        for future in futures:
            result = future.result()
            final_list.extend(result)

    return pd.DataFrame.from_records(final_list).drop_duplicates(subset=['A_team'])


In [ ]:
df = parallel_algorithm(data_players, n_process=16)
df.to_csv('./combination.csv', index=False)

In [ ]:
def build_game_sheet(A, B):
    # Prepare the initial settings
    tmp = ["Jogador_1", "Jogador_2", "Jogador_3", "Jogador_4", "Jogador_5", "Jogador_6", "Jogador_7", "Jogador_8", "Jogador_9", "Jogador_10"]
    tmp2 = A + B
    full_players = dict(zip(tmp,tmp2))
    
    # Remove any previous existing game sheet
    if os.path.exists("./Anygoal_Template.docx"):
        os.remove("./Anygoal_Template.docx")

    # Open the template file
    doc = docx.Document("./template/Anygoal_Template.docx")
    
    # Search any cell with string equal to tmp and replace it with tmp2 values
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                if cell.text in full_players:
                    cell.text = cell.text.replace(cell.text, full_players[cell.text])

    # Save the new game sheet
    doc.save("./Anygoal_Template.docx")
    

def get_best_combination(df):
    # Get the list of values
    mean_smallest = df["Mean_Distance"].nsmallest(3).tolist()
    median_smallest = df["Median_Distance"].nsmallest(3).tolist()
    mean_without_fst_max_smallest = df["Mean_Distance_without_fst_Max"].nsmallest(3).tolist()
    median_without_fst_max_smallest = df["Median_Distance_without_fst_Max"].nsmallest(3).tolist()
    eucDistance = df["Euclidian_Distance"].nsmallest(3).tolist()
    cosSim = df["Cosine_Similarity"].nlargest(3).tolist()
    
    df["Score"] = 0
    for i in range(3):
        mask_mean = df["Mean_Distance"] == mean_smallest[i]
        mask_median = df["Median_Distance"] == median_smallest[i]
        mask_euc = df["Euclidian_Distance"] == eucDistance[i]
        mask_cos = df["Cosine_Similarity"] == cosSim[i]
        mask_mean_without_fst_max = df["Mean_Distance_without_fst_Max"] == mean_without_fst_max_smallest[i]
        mask_median_without_fst_max = df["Median_Distance_without_fst_Max"] == median_without_fst_max_smallest[i]

        df.loc[mask_mean, "Score"] += (5 - 2 * i)
        df.loc[mask_median, "Score"] += (5 - 2 * i)
        df.loc[mask_euc, "Score"] += (5 - 2 * i)
        df.loc[mask_cos, "Score"] += (5 - 2 * i)
        df.loc[mask_mean_without_fst_max, "Score"] += (5 - 2 * i)
        df.loc[mask_median_without_fst_max, "Score"] += (5 - 2 * i)

    highest_score_index = df["Score"].idxmax()
    highest_score = df.loc[highest_score_index, ["A_team", "B_team"]].copy()
    return highest_score["A_team"].split(), highest_score["B_team"].split()

In [ ]:
# Get the A, B teams
A, B = get_best_combination(df)
# Write the game sheet
build_game_sheet(A, B)

In [ ]:
def start_eleven(team, players):
    # Open and create the image object so it can be ready to be edited
    starter = Image.open("./template/five_side.png")
    draw = ImageDraw.Draw(starter) 
    # Font
    font = ImageFont.truetype("./template/newsgott-regular.ttf", 17)

    # Long Names: ST, LW, RW, CB, GR
    text_pos = [(270,190), (140,300), (390,300),(270,375),(270,495)]

    # Short Names: ST, LW, RW, CB, GR
    text_pos_v2 = [(270,190), (140,300), (390,300),(270,375),(270,495)]

    for i in range(0,5):
        # Center players with short names
        if len(players[i]) <= 9:
            draw.text(tuple(map(operator.add, text_pos[i],(25,0))), players[i], font=font)
        else: 
            draw.text(text_pos[i], players[i], font=font)

    if team.upper() == "A": starter.save("./teamA.png")
    else: starter.save("./teamB.png")

In [ ]:
# Output the teams
start_eleven("A", A)
start_eleven("B", B)

In [ ]:
A

In [ ]:
B